In [92]:
## parameters  key and bpm have a least 20% infulence over the songs streams
# predict key and bpm regargding all the other parameters
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from datetime import date
from sklearn import svm


drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/AGH/PythonForMLGoogleColab/MAIN_DATASET_spotify2023.csv')


data.head()
encoder = LabelEncoder()
data["key"] = encoder.fit_transform(data["key"])
data["mode"] = encoder.fit_transform(data["mode"])
released_day_of_year = []

for i in range(len(data.index)):
    day_of_year = date(data['released_year'][i],
                    data['released_month'][i],
                    data['released_day'][i]).timetuple().tm_yday
    released_day_of_year.append(day_of_year)

# add new series to dataframe
data.insert(len(data.columns), "released_day_of_year", released_day_of_year, True)

data = data[["released_day_of_year", 'bpm','key', 'mode', 'danceability_%', 'valence_%', 'energy_%','acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']]


data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,released_day_of_year,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,195,125,2,0,80,89,83,31,0,8,4
1,82,92,3,0,71,61,74,7,0,10,4
2,181,138,7,0,51,32,53,17,0,31,6
3,235,170,0,0,55,58,72,11,0,11,15
4,138,144,0,1,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...
947,307,144,0,0,60,24,39,57,0,8,3
948,294,166,8,0,42,7,24,83,1,12,6
949,307,92,3,0,80,81,67,4,0,8,6
950,293,97,3,0,82,67,77,8,0,12,5


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
X = data.drop(["bpm", "key"], axis = 1)
y = data["bpm"]

train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2
SEED = 42

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=SEED)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=SEED)

print(f"Percent of Train: {train_ratio*100}%, Validation: {validation_ratio*100}% and Test dataset: {test_ratio*100}% ")


Percent of Train: 60.0%, Validation: 20.0% and Test dataset: 20.0% 


In [27]:
X

,released_day_of_year,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,195,0,80,89,83,31,0,8,4
1,82,0,71,61,74,7,0,10,4
2,181,0,51,32,53,17,0,31,6
3,235,0,55,58,72,11,0,11,15
4,138,1,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...
947,307,0,60,24,39,57,0,8,3
948,294,0,42,7,24,83,1,12,6
949,307,0,80,81,67,4,0,8,6
950,293,0,82,67,77,8,0,12,5


In [28]:
y

0      125
1       92
2      138
3      170
4      144
      ... 
947    144
948    166
949     92
950     97
951     90
Name: bpm, Length: 952, dtype: int64

**Linear regressor**

In [34]:
reg = LinearRegression()
reg.fit(x_train, y_train)
y_pred_val = reg.predict(x_val)

rmse = np.sqrt(np.mean((y_pred_val - y_val)**2))

mean = mean_squared_error(y_val, y_pred_val)
print(f"Rmse: {rmse}, Mean:  {mean}")

Rmse: 30.356422374397155, Mean:  921.5123793728003


**Weighted regressor**

In [36]:
reg = LinearRegression()
weights_train = np.ones_like(y_train)
np.random.seed(42)
random_weights = np.random.rand(len(y_train))

reg.fit(x_train, y_train, sample_weight=weights_train)
y_pred_val = reg.predict(x_val)

rmse = np.sqrt(np.mean((y_pred_val - y_val)**2))
mean = mean_squared_error(y_val, y_pred_val)
print(f"Rmse: {rmse}, Mean:  {mean}")

Rmse: 30.356422374397155, Mean:  921.5123793728003


**MLP regressor**

In [77]:
reg = MLPRegressor(hidden_layer_sizes=(4, 6, 5, 2), solver='lbfgs', learning_rate='adaptive')
reg.fit(x_train, y_train)
y_pred_val = reg.predict(x_val)

rmse = np.sqrt(np.mean((y_pred_val - y_val)**2))
mean = mean_squared_error(y_val, y_pred_val)
print(f"Rmse: {rmse}, Mean:  {mean}")

Rmse: 30.938702101923067, Mean:  957.2032877515388


**Polynomial reggresor**

In [90]:
poly_features = PolynomialFeatures(degree = 1)
x_train_poly = poly_features.fit_transform(x_train)
x_train_val = poly_features.fit_transform(x_val)

reg = LinearRegression()
reg.fit(x_train_poly, y_train)

y_pred_val = reg.predict(x_train_val)

rmse = np.sqrt(np.mean((y_pred_val - y_val)**2))
mean = mean_squared_error(y_val, y_pred_val)
print(f"Rmse: {rmse}, Mean:  {mean}")

Rmse: 30.356422374397155, Mean:  921.5123793728003


**SVM**

In [95]:
reg = svm.SVC(kernel='rbf')
reg.fit(x_train, y_train)
y_pred_val = reg.predict(x_val)

rmse = np.sqrt(np.mean((y_pred_val - y_val)**2))
mean = mean_squared_error(y_val, y_pred_val)
print(f"Rmse: {rmse}, Mean:  {mean}")

Rmse: 31.183075706644395, Mean:  972.3842105263158
